In [6]:
import torch
import torch.nn as nn

BATCH_SIZE = 24

randn_a = torch.randn(BATCH_SIZE, 3, 224, 224)
randn_b = torch.randn(BATCH_SIZE, 3, 224, 224)

mse_mean = nn.MSELoss(reduction='mean')
mse_sum = nn.MSELoss(reduction='sum')

print(mse_mean(randn_a, randn_b))
print(mse_sum(randn_a, randn_b))

print(mse_mean(randn_a, randn_b))
print(mse_sum(randn_a, randn_b) / (BATCH_SIZE * 224 * 224 * 3))





tensor(2.0022)
tensor(7233355.5000)
tensor(2.0022)
tensor(2.0022)
